To run this Fenic demo, click **Runtime** > **Run all**.

<div class="align-center">
<a href="https://github.com/typedef-ai/fenic"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/typedef-fenic-logo-github-yellow.png" height="50"></a>
<a href="https://discord.gg/GdqF3J7huR"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/join-the-discord.png" height="50"></a>
<a href="https://docs.fenic.ai/latest/"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/documentation.png" height="50"></a>

Questions? Join the Discord and ask away! For feature requests or to leave a star, visit our [GitHub](https://github.com/typedef-ai/fenic).

</div>

In [ ]:
!pip uninstall -y sklearn-compat ibis-framework imbalanced-learn google-genai
!pip install polars==1.30.0
# === GOOGLE GEMINI ===
#!pip install fenic[google]
# === ANTHROPIC CLAUDE ===
#!pip install fenic[anthropic]
# === OPENAI (Default) ===
!pip install fenic

In [ ]:
import os 
import getpass

# 🔌 MULTI-PROVIDER SETUP - Choose your preferred LLM provider
# Uncomment ONE of the provider sections below:

# === OPENAI (Default) ===
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

# === GOOGLE GEMINI ===
# os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google API Key:")

# === ANTHROPIC CLAUDE ===
# os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Anthropic API Key:")

# 🎤 Meeting Notes Analysis

**Hook:** *"Transform messy meeting transcripts into structured action plans"*

Meeting notes are often scattered, incomplete, and hard to follow up on. "Sarah mentioned something about the deadline... Mike had concerns about budget..." Sound familiar? Watch AI turn rambling meeting transcripts into organized insights with clear owners and deadlines.

**What you'll see in this 2-minute demo:**
- 🎙️ **Raw meeting transcripts** - Unstructured conversation flows
- 📋 **Action item extraction** - Who does what, by when
- 🎯 **Decision tracking** - Key conclusions and commitments
- 📊 **Meeting insights** - Participation, sentiment, follow-ups

Perfect for project management, executive assistance, and team productivity.

In [ ]:
import fenic as fc
from pydantic import BaseModel, Field
from typing import List, Literal

# ⚡ Configure for meeting analysis
session = fc.Session.get_or_create(fc.SessionConfig(
    app_name="meeting_notes_demo",
    semantic=fc.SemanticConfig(
        language_models={
            "meeting_ai": fc.OpenAILanguageModel(model_name="gpt-4o-mini", rpm=500, tpm=200_000),
            # "meeting_ai": fc.GoogleDeveloperLanguageModel(model_name="gemini-2.5-flash-lite", rpm=1000, tpm=1_000_000),
            # "meeting_ai": fc.AnthropicLanguageModel(model_name="claude-3-5-sonnet-20241022", rpm=500, tpm=200_000)
        }
    )
))

print("✅ Meeting notes analysis session configured")

## 📋 Step 1: Meeting Analysis Schema

Define the structured insights we want to extract from meeting transcripts:

In [ ]:
# 📊 Meeting insights extraction schema
class ActionItem(BaseModel):
    task: str = Field(description="Description of the task to be completed")
    owner: str = Field(description="Person responsible for completing the task")
    deadline: str = Field(description="When the task should be completed")

class MeetingInsights(BaseModel):
    meeting_type: Literal["standup", "planning", "review", "decision", "brainstorm", "budget", "quarterly_review"] = Field(description="Type of meeting")
    key_decisions: List[str] = Field(description="Important decisions made during the meeting")
    action_items: List[ActionItem] = Field(description="Tasks with owner and deadline")
    unresolved_issues: List[str] = Field(description="Open questions or concerns raised")
    participants_mentioned: List[str] = Field(description="Names of people mentioned or present")
    next_meeting_needed: bool = Field(description="Whether follow-up meeting is required")
    meeting_sentiment: Literal["positive", "neutral", "tense", "productive", "concerning"] = Field(description="Overall tone of the meeting")
    executive_summary: str = Field(description="One-sentence meeting outcome summary")

print("📊 Meeting Analysis Schema:")
print("   • meeting_type: Automatic meeting classification")
print("   • key_decisions: Important conclusions reached")
print("   • action_items: Tasks with owners and deadlines")
print("   • unresolved_issues: Open questions for follow-up")
print("   • participants_mentioned: Attendee identification")
print("   • next_meeting_needed: Follow-up meeting flag")
print("   • meeting_sentiment: Tone and mood analysis")
print("   • executive_summary: Brief outcome description")

## 🎙️ Step 2: Raw Meeting Transcripts

Realistic meeting transcripts with action items buried in conversation:

In [ ]:
# 🎙️ Meeting transcripts with embedded action items and decisions
meeting_transcripts = session.create_dataframe([
    {
        "meeting_id": "MTG001",
        "meeting_title": "Q1 Product Launch Planning",
        "date": "2024-01-22",
        "transcript": """Sarah: Thanks everyone for joining. Let's discuss the Q1 product launch timeline. Mike, where are we with the API documentation? Mike: About 70% complete. I need another week to finish the authentication section. Sarah: Okay, so target completion by January 29th? Mike: Yes, that works. Lisa: I have concerns about the mobile app testing. We've only tested on iOS 16+, but our user base includes iOS 15 users. Should we expand testing? Sarah: Good point. Lisa, can you coordinate with QA to include iOS 15 testing by February 5th? Lisa: Sure, I'll reach out to Jennifer today. Tom: What about the marketing campaign? We need final product screenshots for the landing page. Sarah: Mike, once API docs are done, can you provide test data for Tom's screenshots? Mike: Absolutely. I'll have sample API responses ready by February 1st. Sarah: Perfect. Any other blockers? Lisa: Just the iOS testing concern, but that's covered now. Sarah: Great, let's reconvene next Tuesday to check progress."""
    },
    {
        "meeting_id": "MTG002", 
        "meeting_title": "Budget Review - Engineering Team",
        "date": "2024-01-18",
        "transcript": """David: We're here to review Q1 engineering budget. Current burn rate is higher than projected. We're at $180K monthly vs budgeted $150K. Emily: The AWS costs are the main issue. Our new ML models are expensive to run. David: How much can we optimize? Emily: I estimate 30-40% savings with better instance management. I need two weeks to implement auto-scaling. David: That's priority one. Emily, please have a cost optimization plan by February 2nd. Also, we need to delay the new feature development until costs are under control. Mark: What about the hiring plans? We have three open engineering positions. David: Let's pause hiring until we see the cost improvements. Mark, can you put those requisitions on hold? Mark: I already sent offers to two candidates. David: Let's honor those offers but pause the third position. This is tough but necessary. Emily: I understand the pressure. The optimization work is my top priority. David: Thanks everyone. This isn't ideal but we'll get through it. Let's meet again in two weeks to review progress."""
    },
    {
        "meeting_id": "MTG003",
        "meeting_title": "Customer Success Team Standup", 
        "date": "2024-01-19",
        "transcript": """Anna: Good morning team. Let's go around for updates. Jessica, how are the enterprise onboardings going? Jessica: Really well! TechFlow Inc. is live and loving the analytics dashboard. GlobalCorp starts Monday. I do need help with their custom integration though. Anna: What kind of help? Jessica: They need a custom webhook for their CRM. It's beyond my technical scope. Anna: Let me connect you with the engineering team. I'll set up a meeting with Mike for next Wednesday. Rachel: I have an update on the customer satisfaction survey. Response rate is only 12%, way below our 25% target. Anna: Hmm, that's concerning. What do you think is causing the low response? Rachel: The survey is too long - 15 questions. Customers are abandoning it. Anna: Good insight. Rachel, can you create a shorter version with just 5 key questions by end of week? Rachel: Definitely. I'll focus on the most important metrics. Jessica: Also, GlobalCorp mentioned they'd be interested in our premium support tier. Anna: Excellent! Jessica, follow up with them about pricing after their integration is complete. Rachel: Should we track these premium support inquiries somewhere? Anna: Yes, add it to our CRM pipeline. Meeting same time Monday, everyone."""
    },
    {
        "meeting_id": "MTG004",
        "meeting_title": "Quarterly Review - Executive Team",
        "date": "2024-01-25", 
        "transcript": """CEO Sarah: Thank you all for a solid Q4. Revenue hit $2.1M, 15% above target. CFO Michael: The growth is encouraging but we need to watch cash flow. Current runway is 14 months at this burn rate. CTO Lisa: Engineering team delivered all major features on time. The new AI capabilities are getting great customer feedback. CEO Sarah: Lisa, what's our technical debt situation? CTO Lisa: Manageable but growing. I recommend dedicating one engineer full-time to refactoring in Q2. CFO Michael: What's the cost impact? CTO Lisa: About $120K in salary costs, but it'll prevent much larger problems later. CEO Sarah: Approved. Lisa, identify the engineer and start in April. VP Sales Tom: Pipeline looks strong for Q1. We have $800K in qualified opportunities. CEO Sarah: Tom, what's your confidence on closing rate? VP Sales Tom: 65% based on current conversations. One concern - two major prospects asked about SOC2 compliance. CEO Sarah: We need to prioritize that. Michael, can you research SOC2 compliance costs? CFO Michael: I'll get quotes from auditors and have recommendations by February 15th. CEO Sarah: Perfect. Any other strategic issues? CTO Lisa: We should consider expanding to European markets, but GDPR compliance is complex. CEO Sarah: Let's table that for Q2 planning. Good meeting everyone."""
    }
])

print("🎙️ Meeting Transcripts - Notice the buried action items and decisions:")
meeting_transcripts.select("meeting_id", "meeting_title", "date").show()

## 🧠 Step 3: Intelligent Meeting Analysis

Extract structured insights from conversational meeting transcripts:

In [ ]:
# 🧠 Extract meeting insights from transcripts
meeting_analysis = meeting_transcripts.select(
    "meeting_id",
    "meeting_title", 
    "date",
    fc.semantic.extract(
        "transcript",
        MeetingInsights,
        model_alias="meeting_ai"
    ).alias("insights")
).cache()

# Show meeting types and sentiment analysis
meeting_overview = meeting_analysis.select(
    "meeting_id",
    "meeting_title",
    meeting_analysis.insights.meeting_type.alias("type"),
    meeting_analysis.insights.meeting_sentiment.alias("sentiment"),
    meeting_analysis.insights.executive_summary.alias("summary")
)

print("🧠 MEETING ANALYSIS OVERVIEW:")
meeting_overview.show()

## 📋 Step 4: Action Item Extraction

Extract actionable tasks with owners and deadlines from meeting conversations:

In [ ]:
# 📋 Extract action items and decisions
action_tracking = meeting_analysis.select(
    "meeting_id",
    "meeting_title",
    meeting_analysis.insights.action_items.alias("action_items"),
    meeting_analysis.insights.key_decisions.alias("decisions"),
    meeting_analysis.insights.unresolved_issues.alias("open_issues"),
    meeting_analysis.insights.next_meeting_needed.alias("followup_needed")
)

print("📋 ACTION ITEMS AND DECISIONS:")
action_tracking.show()

# Meeting productivity analysis
total_meetings = meeting_transcripts.count()
productive_meetings = meeting_analysis.filter(
    meeting_analysis.insights.meeting_sentiment.is_in(["positive", "productive"])
).count()
followup_needed = meeting_analysis.filter(meeting_analysis.insights.next_meeting_needed).count()

print("\n📊 MEETING PRODUCTIVITY INSIGHTS:")
print(f"   • Total meetings analyzed: {total_meetings}")
print(f"   • Productive/positive meetings: {productive_meetings} ({productive_meetings/total_meetings*100:.0f}%)")
print(f"   • Meetings requiring follow-up: {followup_needed}")

print("\n🎯 BUSINESS BENEFITS:")
print("   • Automatic action item extraction with owners and deadlines")
print("   • Decision tracking prevents 'what did we decide?' confusion")
print("   • Sentiment analysis identifies team dynamics issues")
print("   • Follow-up meeting identification improves project continuity")
print("   • Participant mentions enable better communication planning")
print("   • Unresolved issues tracking prevents items from falling through cracks")

In [ ]:
session.stop()